In [1]:
print("Settings.py should be e.g. cnn_flows_prtrain")
#DESIGN PARAMETERS FOR NEURAL NETWORK

VALIDATION_SPLIT = 0.0 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

TRAINING_PREPARATION = False

BATCH_SIZE = 5
TIME_STEPS = 10

NR_EPOCHS = 100

LR = 0.0001
MOMENTUM = 0.9
print("Hyperparameters defined")
print("Bug with learning rate when sequence length is not the same for each element of the dataset")

Settings.py should be e.g. cnn_flows_prtrain
Hyperparameters defined
Bug with learning rate when sequence length is not the same for each element of the dataset


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import time
import os
import os.path as path
import copy

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer
from gait_analysis import WeightWatcher

import matplotlib.pyplot as plt

from gait_analysis.Models import PretrainConvFlow

print("done")

done


In [3]:
#change configuration in settings.py
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

loading configuration  cnn_flows_pretrain
[OK]
Dataset size: 30
Indices size: 30
Split: 0


In [4]:
if TRAINING_PREPARATION:
    print('Start training...')
    print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
    running_loss = 0.0
    print("Batch size:", BATCH_SIZE)
    print("Evaluating first element...")
    start_time = time.time()
    i, batch = next(iter(enumerate(train_loader)))
    inputs, labels = batch
    #TODO make list disappear

    data_in_list = [s.to(device) for s in inputs['flows']]
    labels = labels.to(device)

    print("--------------------------\nOriginal input:")
    print("Data in: List length", len(data_in_list), "List element:", data_in_list[0].size())
    print("Labels:", labels.size())
    print("--------------------------\nSingle element:")
    data_element = data_in_list[0]
    label_element = labels[:,0].long()
    print("Data element:", data_element.size())
    print("Label element:", label_element.size())

    optimizer.zero_grad() 
    output_element = net(data_element)

    loss = criterion(output_element,label_element)
    loss.backward() 
    optimizer.step()

    running_loss += loss.data.item()
    elapsed_time = time.time() - start_time;
    print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
    print("Time needed:{}s".format(elapsed_time))
    print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
    print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)*TIMESTEPS/BATCH_SIZE/60))
    print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*TIMESTEPS/BATCH_SIZE*NR_EPOCHS/3600))
print("done")

done


In [5]:
net = PretrainConvFlow.PretrainConvFlow()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM)

learning_rate_array = []
loss_array = []
monitor = WeightWatcher(net,['conv5','fc1'])

cpu


In [12]:
print("Start training...")
for epoch in range(NR_EPOCHS): 
    start_time = time.time()
#     print("Epoch:", epoch)
    running_loss = 0.0
    incorrectly_predicted = 0 
    total_elements = 0
    correctly = [0, 0, 0]
    total_class = [0, 0, 0]
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in_list = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        
#         print("--------------------------\nOriginal input:")
#         print("Data in: List length", len(data_in_list), "List element:", data_in_list[0].size())
#         print("Labels:", labels.size())

        for j, data_element in enumerate(data_in_list):
            if (labels.size(1) != len(data_in_list)):
                print("WARNING, labels and data_element length do not match in Set",i, "element", j)
#                 print("Label element after")
                print(labels[0:len(data_in_list),j].size())
            label_element = labels[0:len(data_in_list),j].long()
#             print("--------------------------\nSingle element:")
#             print("Data element size:", data_element.size())
#             print("Label element size:", label_element.size())
#             print("Data element", data_element)
#             print("Label element", label_element)
#             print("Label element after",label_element)

#             print("Data element", data_element[0][0])
    
            optimizer.zero_grad() 
            output_element = net(data_element)
#             print("Output element",output_element)
#             print("Label element after",label_element)
            loss = criterion(output_element,label_element)
            loss.backward() 
            optimizer.step()
            
            running_loss += loss.data.item()/len(dataset)/TIME_STEPS*BATCH_SIZE
#             print("losses:",loss.data.item(),len(dataset),TIMESTEPS)
#             print("Prediction:", torch.max(output_element,1)[1])
#             print("Labels:", label_element.long())
            prediction = torch.max(output_element,1)[1]
            incorrectly_predicted += torch.nonzero(torch.max(output_element,1)[1] - label_element.long()).numel()
            total_elements += (torch.max(output_element,1)[1] - label_element.long()).numel()
            
            for pred,label in zip(prediction,label_element):
                if label == 0:
                    total_class[0] += 1
                    if pred == 0:
                        correctly[0] += 1
                elif label == 1:
                    total_class[1] += 1
                    if pred == 1:
                        correctly[1] += 1
                else:
                    total_class[2] += 1
                    if pred == 2:
                        correctly[2] += 1
    
    print("-----------------------------")
    print("Epoch {}: Loss {}, Acc {}, took {}s".format(epoch, running_loss,1 - incorrectly_predicted/total_elements, time.time()-start_time))
    print("Accuracy by class", [c/(t) for c,t in zip(correctly,total_class)], "Total elements:", total_class)
    monitor.update_weights(net,['conv5','fc1'])
    w_conv5, w_fc1 = monitor.get_weight_changes(['conv5','fc1'])
#     print("FC1:",w_fc1[-1],"CONV5:",w_conv5[-1])
    monitor.update_loss(running_loss)
    learning_rate_array.append(LR)

#test_all_preds(test_net) 
print('...Training finished')
# plt.plot(loss_array)

Start training...
-----------------------------
Epoch 0: Loss 0.34933065593087337, Acc 0.8466666666666667, took 17.372836112976074s
Accuracy by class [0.9328358208955224, 0.8275862068965517, 0.75] Total elements: [134, 58, 108]
-----------------------------
Epoch 1: Loss 0.3297964401519873, Acc 0.8566666666666667, took 19.39310336112976s
Accuracy by class [0.9402985074626866, 0.8448275862068966, 0.7592592592592593] Total elements: [134, 58, 108]
-----------------------------
Epoch 2: Loss 0.3071645486700163, Acc 0.8533333333333333, took 20.128742694854736s
Accuracy by class [0.9402985074626866, 0.8448275862068966, 0.75] Total elements: [134, 58, 108]
-----------------------------
Epoch 3: Loss 0.29553152236350483, Acc 0.8533333333333333, took 20.58406972885132s
Accuracy by class [0.9402985074626866, 0.8448275862068966, 0.75] Total elements: [134, 58, 108]
-----------------------------
Epoch 4: Loss 0.323293677854042, Acc 0.8433333333333333, took 19.886809587478638s
Accuracy by class [0

In [26]:
total = sum([0, 1, 10])
print(total)

11


In [14]:
# torch.save(net.state_dict(), '/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/first_draft')
# Later:
# loaded_model = PretrainConvFlow.PretrainConvFlow()
# loaded_model.load_state_dict(torch.load('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/first_draft'))

In [16]:
# Later:
# loaded_model = PretrainConvFlow.PretrainConvFlow()
# loaded_model.load_state_dict(torch.load('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/first_draft'))

In [20]:
# print(loaded_model.parameters)


<bound method Module.parameters of PretrainConvFlow(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(16, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=90, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=3, bias=True)
)>


In [9]:
plt.plot(monitor.get_weight_changes(['conv5']))
print(monitor.get_weight_changes(['conv5']))
plt.plot(monitor.get_loss())
print(monitor.get_loss())

[1.2156862e-09, 7.847238e-09, 2.0700796e-08, 3.871344e-08, 6.1552804e-08, 8.969565e-08, 1.2218719e-07, 1.5831262e-07, 1.9804826e-07, 2.3933092e-07, 2.8222172e-07, 3.2581772e-07, 3.706511e-07, 4.1725164e-07, 4.6413348e-07, 5.1167103e-07, 5.639011e-07, 6.177802e-07, 6.7190126e-07, 7.2838617e-07, 7.8659906e-07, 8.477239e-07, 9.113527e-07, 9.772217e-07, 1.0441237e-06, 1.1163859e-06, 1.1909921e-06, 1.2679316e-06, 1.3480918e-06, 1.4330559e-06, 1.5212614e-06, 1.6134283e-06, 1.7127483e-06, 1.8157624e-06, 1.9242343e-06, 2.0388193e-06, 2.167997e-06, 2.3079715e-06, 2.4548306e-06, 2.6118473e-06, 2.7824285e-06, 2.9723828e-06, 3.1879845e-06, 3.4336956e-06, 3.7198172e-06, 4.049786e-06, 4.447429e-06, 4.924816e-06, 5.4968773e-06, 6.20003e-06, 7.0652673e-06, 8.143118e-06, 9.50244e-06, 1.1229457e-05, 1.3462059e-05, 1.6277316e-05, 1.9779329e-05, 2.3899389e-05, 2.8418886e-05, 3.338968e-05, 3.9200368e-05, 4.5841312e-05, 5.25605e-05, 5.8519985e-05, 6.375299e-05, 6.876483e-05, 7.3768744e-05, 7.8768426e-05, 8.

In [10]:
# def calc_conv_accuracy(model):

#     #Time for printing
#     testing_start_time = time.time()

#     print('Start testing...')
#     correct = 0 
#     total = 0
#     with torch.no_grad():
#         for i, batch in enumerate(train_loader):
#             inputs, labels = batch
            
#             data_in = [s.to(device) for s in inputs['flows']]
#             labels = labels.to(device)
#             outputs = model(data_in)
# #             print("Out:", len(outputs), outputs.size())
# #             print("Labels:", len(labels), labels.size())
#             _, predicted = torch.max(outputs.data, 1)
# #             print('predicted:',len(predicted),predicted.size())
#             n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
#             total += predicted.numel()
#             # print('predicted',predicted)
#             correct += predicted.numel() - n_errors
#             # print('labels',labels)
#     print('Accuracy {:.3f}%'.format(correct/total))
#     print('...testing finished')
# print("Definition done")